In [1]:
from src.modules.bear_cleaner import *
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import Ridge
from sklearn.svm import SVR
from sklearn.impute import SimpleImputer, KNNImputer, MissingIndicator
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from scipy.optimize import minimize_scalar, minimize, basinhopping
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB, MultinomialNB

from prettytable import PrettyTable
import numpy as np

In [2]:
df = pl.read_json("./data/vins.json")
df = super_pipe(df)
df = df.filter(pl.col("unit_price").is_not_null())
df.head()

name,capacity,unit_price,offer_price,price_bundle,characteristics,keyword_1,keyword_2,keyword_3,picture,millesime,cepage,gouts,par_gouts,oeil,nez,bouche,temp_low,temp_high,service,accords_vins,accords_reco,avg_temp,conservation_date,bio,customer_fav,is_new,top_100,destock,sulphite_free,alcohol_volume,type,country,bubbles,wine_note,nb_reviews
str,f64,f64,f64,str,str,str,str,str,str,i64,str,str,str,str,str,str,f64,f64,str,str,str,f64,i64,i32,i32,i32,i32,i32,i32,f64,str,str,i32,f64,i64
"""ABYMES 2021 - …",0.75,9.5,null,"""7,60 € par 2""","""Vin Blanc / Sa…","""Intense""","""Minéral""","""Superbe""","""https://www.vi…",2021,"""Jacquère""","""Blanc Sec soup…","""Fruité""","""Robe or blanc …","""Des arômes de …","""L'attaque est …",8.0,10.0,"""En bouteille""","""Apéritif, Entr…","""Plateau de fru…",9.0,2023,0,0,0,0,0,0,11.5,"""Vin Blanc""","""France""",0,15.9,10
"""PINOT NOIR 201…",0.75,94.9,null,null,"""Vin Rouge / Ch…","""Elégance""","""Finesse""","""Harmonie""","""https://www.vi…",2019,"""Pinot noir""","""Rouge Charnu e…","""Puissant""","""Robe rubis aux…","""Nez complexe s…","""Bouche fruitée…",8.0,10.0,"""En bouteille o…","""Apéritif, Entr…","""Gigot d'agneau…",9.0,2026,0,0,0,0,0,0,13.5,"""Vin Rouge""","""Chili""",0,null,0
"""SAUMUR BLANC T…",0.75,11.7,null,"""9,90 € par 2""","""Vin Blanc / Lo…","""Fraîcheur""","""100% Chenin""","""Équilibre""","""https://www.vi…",2022,"""Chenin""","""Blanc Sec soup…","""Minéral""","""Couleur jaune …","""Arômes de frui…","""Frais et minér…",6.0,8.0,"""En bouteille""","""Apéritif, Entr…","""Apéritifs, poi…",7.0,2027,0,1,0,0,0,0,13.0,"""Vin Blanc""","""France""",0,17.2,10
"""CHÂTEAU BELLE …",0.75,16.95,null,"""14,40 € par 2""","""Vin Rouge / Bo…","""Fraîcheur""","""Elégant""","""Savoureux""","""https://www.vi…",2016,"""Merlot""","""Rouge Puissant…","""Puissant""","""Robe violette,…","""Bleuets sauvag…","""Fraîcheur, élé…",16.0,18.0,"""Ouvrir avant l…","""Apéritif, Char…",null,17.0,2030,0,0,0,0,0,0,13.5,"""Vin Rouge""","""France""",0,null,0
"""MORGON - LE CL…",0.75,25.5,null,null,"""Vin Rouge / Be…","""Nature""","""Savoureux""","""Vigneron""","""https://www.vi…",2019,"""Gamay""","""Rouge Léger et…","""Fruité""","""Rouge vif.""","""Des notes de c…","""Profond, équil…",13.0,16.0,"""En bouteille""","""Apéritif, Char…","""Poularde de Br…",14.5,2032,0,0,0,0,0,0,13.0,"""Vin Rouge""","""France""",0,null,0


models = train_model(X_train, y_train)

model_result(knn = models["model_knn"], rf = models["model_rf"],
             boost = models["model_boost"], ridge = models["model_ridge"], 
             svm = models["model_svm"], mlp = models["model_mlp"])

model_param(models["model_knn"], "entrainement__n_neighbors")

model_result(knn=model_knn(X_train,y_train))

In [3]:
from sklearn.preprocessing import LabelEncoder
from src.modules.models import *

In [5]:
df = prep_str(df)

In [6]:
X = df[["avg_temp", "conservation_date", "bio", "customer_fav", "top_100", "destock", "sulphite_free","alcohol_volume", 
        "bubbles", "wine_note", "keyword_1","keyword_2","keyword_3","cepage","par_gouts", "service","type","country"]]
y = df["unit_price"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [7]:
X_train.head()

,avg_temp,conservation_date,bio,customer_fav,top_100,destock,sulphite_free,alcohol_volume,bubbles,wine_note,keyword_1,keyword_2,keyword_3,cepage,par_gouts,service,type,country
2008,7.5,2024.0,0,1,0,0,0,12.0,0,15.8,33,573,473,132,4,0,2,14
2506,13.5,2030.0,0,0,0,0,0,13.0,0,NaN,204,453,549,36,4,1,0,14
707,16.5,2029.0,0,0,0,0,0,13.5,0,NaN,164,149,718,154,6,3,2,14
1045,17.0,2075.0,0,0,0,0,0,13.0,0,NaN,226,237,1426,27,6,4,2,14
2545,13.0,2031.0,0,0,0,0,0,13.5,0,NaN,146,609,325,36,6,2,0,14


models = train_model(X_train, y_train)

model_result(knn = models["model_knn"], rf = models["model_rf"],
             boost = models["model_boost"], ridge = models["model_ridge"], 
             svm = models["model_svm"], mlp = models["model_mlp"])

In [8]:
model = model_boost(X_train, y_train)

c:\Users\guill\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
2400 fits failed out of a total of 10800.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1200 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\guill\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\guill\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py", line 1152, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\guill

In [12]:
model_result(boost=model)

+--------+-------+------+
| Modèle | Score |  SD  |
+--------+-------+------+
| boost  | 0.305 | 0.06 |
+--------+-------+------+


In [14]:
model_param(model, 
            "imputation__strategy",
            "entrainement__learning_rate",
            
            "entrainement__alpha",
            "entrainement__loss")

+-----------------------------+---------------+
|          Parameter          |     Value     |
+-----------------------------+---------------+
|     imputation__strategy    | most_frequent |
| entrainement__learning_rate |      0.01     |
|     entrainement__alpha     |      0.75     |
|      entrainement__loss     |    quantile   |
+-----------------------------+---------------+
